In [1]:
import json
import re


In [3]:
def get_conversation(content):
    messages = []
    tags = []
    for turn in content:
        messages.append(turn['message'])
        tags.append(turn['mezza_da'])
    messages = "".join(messages)
    messages = messages.replace(" ", "").lower()
    messages = messages.replace("\n", "")
    messages = messages.replace("\t", "")
    messages = re.sub('\W+','', messages)
    return "".join(messages), tags

In [4]:
tagged_conv = []
taggs = []
tagged = ["tc_processed/train_full_anno.json", "tc_processed/test_freq_full_anno.json", 
          "tc_processed/test_rare_full_anno.json", "tc_processed/valid_freq_full_anno.json", 
          "tc_processed/valid_rare_full_anno.json"]
for each in tagged:
    with open(each) as f:
        data = json.load(f)
    keys = list(data.keys())
    for each in keys:
        content = data[each]['content']
        c, t = get_conversation(content)
        tagged_conv.append(c)
        taggs.append(t)

In [82]:
non = "processed_output/test_freq.src"
with open(non) as f:
    src = f.readlines()
non_tgt = "processed_output/test_freq.tgt"
with open(non_tgt) as f:
    tgt = f.readlines()

In [83]:
original = []
for i in range(len(src)-1):
    if src[i].replace("\n", "") not in src[i+1]:
        original.append(src[i])

In [84]:
processed = []
for i in range(len(src)):
    text = src[i]
    text = text.replace("_eos", "")
    text = text.replace("\n", "")
    text = text.replace(" ", "")
    text = re.sub('\W+', '', text)
    processed.append(text)
conversations = []
for i in range(len(processed)-1):
    if processed[i] not in processed[i+1]:
        ending = tgt[i].replace("\n", "")
        ending = ending.replace(" ", "")
        ending = ending.replace("_eos", "")
        ending = ending.replace("_go", "")
        ending = re.sub('\W+', '', ending)
        c = processed[i] + ending
        conversations.append(c)
ending = tgt[len(processed)-1].replace("\n", "")
ending = ending.replace(" ", "")
ending = ending.replace("_eos", "")
ending = ending.replace("_go", "")
ending = re.sub('\W+', '', ending)
c = processed[len(processed)-1] + ending
conversations.append(c)

In [85]:
not_in =[]
for i in range(len(conversations)):
    if conversations[i] not in tagged_conv:
        not_in.append(i)

In [86]:
not_in

[]

In [87]:
for each in not_in:
    print(each)

In [88]:
src.index(original[385])

8031

In [89]:
not_in_convs = []
for each in not_in:
    current_idx = src.index(original[each])
    previous = src[current_idx - 1].replace("\n", "")
    current = src[current_idx].replace("\n", "")
    c = [current]
    while previous in current:
        c.insert(0, previous)
        current_idx -= 1
        current = previous
        previous = src[current_idx - 1].replace("\n", "")
    not_in_convs.append(c)
    

In [90]:
not_in_convs[0]

IndexError: list index out of range

In [91]:
for each in not_in_convs:
    print(len(each))

In [92]:
for each in not_in_convs:
    print(each[0])

In [93]:
ids = []
for i in range(len(conversations)):
    if conversations[i] in tagged_conv:
        ids.append(tagged_conv.index(conversations[i]))
    else:
        ids.append("NA")

In [94]:
def get_taggs(tag_list):
    output = ""
    for i in range(len(tag_list)):
        for each in tag_list[i]:
            output += f"{each['da']} "
        if i != len(tag_list) - 1:
            output += "_eos "
    return output[:-1]
        
        

In [95]:
output = []
na_count = 0
untagged_len = []
tagged_len = []
for conv in ids:
    if conv != "NA":
        untagged_len.append(len(taggs[conv])-1)
        conv_tag = taggs[conv]
        for i in range(1, len(conv_tag)):
            output.append(get_taggs(conv_tag[:i]))
    else:
        untagged_len.append(len(not_in_convs[na_count]))
        output = output + not_in_convs[na_count]
        na_count +=1 


In [96]:
with open('processed_output/test_freq.src.da', 'w') as filehandle:
    filehandle.writelines("%s\n" % place for place in output)